En este archivo puedes escribir lo que estimes conveniente. Te recomendamos detallar tu solución y todas las suposiciones que estás considerando. Aquí puedes ejecutar las funciones que definiste en los otros archivos de la carpeta src, medir el tiempo, memoria, etc.

In [10]:
from typing import List, Tuple
import pandas as pd
from datetime import datetime
pd.set_option('display.max_columns', None)

In [11]:
data_path = '../data/data.parquet'
df = pd.read_parquet(data_path)
df.head()

,url,date,content,renderedContent,id,user_username,user_displayname,user_id,user_description,user_rawDescription,user_verified,user_created,user_followersCount,user_friendsCount,user_statusesCount,user_favouritesCount,user_listedCount,user_mediaCount,user_location,user_protected,user_linkUrl,user_linkTcourl,user_profileImageUrl,user_profileBannerUrl,user_url,outlinks,tcooutlinks,replyCount,retweetCount,likeCount,quoteCount,conversationId,lang,source,sourceUrl,sourceLabel,media_thumbnailUrl,media_variants_contentType,media_variants_url,media_variants_bitrate,media_duration,media_type,retweetedTweet,quotedTweet_url,quotedTweet_date,quotedTweet_content,quotedTweet_renderedContent,quotedTweet_id,quotedTweet_user_usernameeet_,quotedTweet_user_displayname,quotedTweet_user_id,quotedTweet_user_description,quotedTweet_user_rawDescription,quotedTweet_user_verified,quotedTweet_user_created,quotedTweet_user_followersCount,quotedTweet_user_friendsCount,quotedTweet_user_statusesCount,quotedTweet_user_favouritesCount,quotedTweet_user_listedCount,quotedTweet_user_mediaCount,quotedTweet_user_location,quotedTweet_user_protected,quotedTweet_user_linkUrl,quotedTweet_user_linkTcourl,quotedTweet_user_profileImageUrl,quotedTweet_user_profileBannerUrl,quotedTweet_user_url,quotedTweet_replyCount,quotedTweet_retweetCount,quotedTweet_likeCount,quotedTweet_quoteCount,quotedTweet_conversationId,quotedTweet_lang,quotedTweet_source,quotedTweet_sourceUrl,quotedTweet_sourceLabel,quotedTweet_retweetedTweet,quotedTweet_outlinks,quotedTweet_tcooutlinks,quotedTweet_media_previewUrl,quotedTweet_media_fullUrl,quotedTweet_media_type,quotedTweet_mentionedUsers_username,quotedTweet_mentionedUsers_displayname,quotedTweet_mentionedUsers_id,quotedTweet_mentionedUsers_description,quotedTweet_mentionedUsers_rawDescription,quotedTweet_mentionedUsers_verified,quotedTweet_mentionedUsers_created,quotedTweet_mentionedUsers_followersCount,quotedTweet_mentionedUsers_friendsCount,quotedTweet_mentionedUsers_statusesCount,quotedTweet_mentionedUsers_favouritesCount,quotedTweet_mentionedUsers_listedCount,quotedTweet_mentionedUsers_mediaCount,quotedTweet_mentionedUsers_location,quotedTweet_mentionedUsers_protected,quotedTweet_mentionedUsers_linkUrl,quotedTweet_mentionedUsers_linkTcourl,quotedTweet_mentionedUsers_profileImageUrl,quotedTweet_mentionedUsers_profileBannerUrl,quotedTweet_mentionedUsers_url,mentionedUsers_username,mentionedUsers_displayname,mentionedUsers_id,mentionedUsers_description,mentionedUsers_rawDescription,mentionedUsers_verified,mentionedUsers_created,mentionedUsers_followersCount,mentionedUsers_friendsCount,mentionedUsers_statusesCount,mentionedUsers_favouritesCount,mentionedUsers_listedCount,mentionedUsers_mediaCount,mentionedUsers_location,mentionedUsers_protected,mentionedUsers_linkUrl,mentionedUsers_linkTcourl,mentionedUsers_profileImageUrl,mentionedUsers_profileBannerUrl,mentionedUsers_url
0,https://twitter.com/ArjunSinghPanam/status/136...,2021-02-24 09:23:35,The world progresses while the Indian police a...,The world progresses while the Indian police a...,1364506249291784198,ArjunSinghPanam,Arjun Singh Panam,45091142,"Global Citizen, Actor, Director: Sky is the ro...","Global Citizen, Actor, Director: Sky is the ro...",False,2009-06-06 07:50:57,603,311,17534,4269,23,1211,,False,https://www.cosmosmovieofficial.com,https://t.co/3uaoV3gCt3,https://pbs.twimg.com/profile_images/121554174...,https://pbs.twimg.com/profile_banners/45091142...,https://twitter.com/ArjunSinghPanam,https://twitter.com/ravisinghka/status/1364150...,https://t.co/es3kn0IQAF,0,0,0,0,1364506249291784198,en,"<a href=""http://twitter.com/download/iphone"" r...",http://twitter.com/download/iphone,Twitter for iPhone,None,None,None,None,None,None,None,https://twitter.com/RaviSinghKA/status/1364150...,2021-02-23T09:51:20+00:00,"This is what the indian police are good at, be...","This is what the indian police are good at, be...",1364150844757860352,RaviSinghKA,ravinder singh,2347762888,"Founder/C

In [13]:
from q1_time import q1_time

file_path = '../data/data.parquet'
q1_time(file_path)

[(datetime.date(2021, 2, 12), 'RanbirS00614606'),
 (datetime.date(2021, 2, 13), 'MaanDee08215437'),
 (datetime.date(2021, 2, 17), 'RaaJVinderkaur'),
 (datetime.date(2021, 2, 16), 'jot__b'),
 (datetime.date(2021, 2, 14), 'rebelpacifist'),
 (datetime.date(2021, 2, 18), 'neetuanjle_nitu'),
 (datetime.date(2021, 2, 15), 'jot__b'),
 (datetime.date(2021, 2, 20), 'MangalJ23056160'),
 (datetime.date(2021, 2, 23), 'Surrypuria'),
 (datetime.date(2021, 2, 19), 'Preetm91')]

In [14]:
from q1_memory import q1_memory

file_path = '../data/data.parquet'
q1_memory(file_path)

[(datetime.date(2021, 2, 12), 'RanbirS00614606'),
 (datetime.date(2021, 2, 13), 'MaanDee08215437'),
 (datetime.date(2021, 2, 17), 'RaaJVinderkaur'),
 (datetime.date(2021, 2, 16), 'jot__b'),
 (datetime.date(2021, 2, 14), 'rebelpacifist'),
 (datetime.date(2021, 2, 18), 'neetuanjle_nitu'),
 (datetime.date(2021, 2, 15), 'jot__b'),
 (datetime.date(2021, 2, 20), 'MangalJ23056160'),
 (datetime.date(2021, 2, 23), 'Surrypuria'),
 (datetime.date(2021, 2, 19), 'Preetm91')]

In [56]:
from q2_memory import q2_memory

file_path = '../data/data.parquet'
q2_memory(file_path)

[('🙏', 7286),
 ('😂', 3072),
 ('🚜', 2972),
 ('|', 2429),
 ('✊', 2411),
 ('🌾', 2363),
 ('🇮', 2096),
 ('🇳', 2094),
 ('❤', 1779),
 ('🤣', 1668)]